In [5]:
import pandas as pd
import nltk, re, string
import numpy as np
from scipy import sparse

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from sklearn import svm
from nltk.corpus import stopwords

from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
import itertools
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cross_validation import StratifiedKFold
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re, collections
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import words as w

from nltk.corpus import *
from nltk.collocations import *
from string import punctuation
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer

/home/nihar/anaconda3/envs/Info256/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/nihar/anaconda3/envs/Info256/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/home/nihar/anaconda3/envs/Info256/lib/python3.5/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("T

In [6]:
def build_tokens(text):
    tweetTokenizer = nltk.tokenize.TweetTokenizer(strip_handles=True, reduce_len=True)
    tokens = tweetTokenizer.tokenize(text)
    #tokens = preprocess(text, lowercase=True)
    #tokens = [nltk.WordNetLemmatizer().lemmatize(token) for token in tokens]
    #tokens= [nltk.PorterStemmer().stem(token) for token in tokens]
    return tokens


In [7]:
class AdditionalFeatureExtractor(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def get_feature_names(self):
        return (['percent_bad','vader_compound','num_words','vader_neg','vader_pos'])
    
    def num_bad(self, df):
        #get number of words in each sentence
        num_words = [len(word) for word in df]
        
        #get percent of abusive words in each sentence
        with open("list_of_abuses.txt", "r") as abuse_list:
            abuses = abuse_list.read().split()
            num_abuses = 0
            for abuse in abuses:
                num_abuses += 1
            # number of badwords in list of abuses
            num_bad = [np.sum([word.lower().count(abuse) for abuse in abuses])
                                                for word in df]
            norm_bad = np.array(num_bad) / np.array(num_words, dtype=np.float)
        return norm_bad
    
    def num_words(self,df):
        #get number of words in each sentence
        num_words = [len(word) for word in df]
        return num_words
    
    def vader_helper(self, df):
        #vader analysis
        sid = SentimentIntensityAnalyzer()
        vader_feature = []
        for sentence in df:
            ss = sid.polarity_scores(sentence)
            vader_feature.append(ss['compound'])
        return vader_feature
    
    def vader_helper_neg(self, df):
        #vader analysis
        sid = SentimentIntensityAnalyzer()
        vader_feature = []
        for sentence in df:
            ss = sid.polarity_scores(sentence)
            vader_feature.append(ss['neg'])
        return vader_feature
    
    def vader_helper_pos(self, df):
        #vader analysis
        sid = SentimentIntensityAnalyzer()
        vader_feature = []
        for sentence in df:
            ss = sid.polarity_scores(sentence)
            vader_feature.append(ss['pos'])
        return vader_feature
    def transform(self, df, y=None):     
        #add both the features to an array
        X = np.array([self.num_bad(df), self.vader_helper(df),self.num_words(df),self.vader_helper_neg(df),self.vader_helper_pos(df)]).T
        #X = np.array([self.num_bad(df),self.vader_helper(df)]).T
        #X.reshape(-1, 1) #use if only 1 feature
        if not hasattr(self, 'scalar'):
            self.scalar = preprocessing.StandardScaler().fit(X)
        return self.scalar.transform(X)      

    def fit(self, df, y=None):
        return self

In [8]:
stopwords = ['a','an','the','or','is','are','was','were','have']
def all_features():
    features = []
    custom_features = AdditionalFeatureExtractor() # this class includes my custom features 
    vect = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,6), analyzer= "char", stop_words = stopwords, tokenizer= build_tokens)
    vect1 = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,6), analyzer= "word", stop_words = stopwords, tokenizer= build_tokens)
    
    features.append(('ngram', vect))
    features.append(('ngram1', vect1))
   
    features.append(('custom_features', custom_features))
    return features



In [17]:
import pickle

#Load the model from this pickle later
best_clf = pickle.load(open('model.pkl','rb'))

# returns confidence score ()
def is_abuse(model,sentence):
    confidence = model.predict_proba([sentence])
    print(confidence)
    return confidence


sentence = "you fuck!"
# calling the function
is_abuse(best_clf,sentence)

/home/nihar/anaconda3/envs/Info256/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version pre-0.18 when using version 0.18. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/nihar/anaconda3/envs/Info256/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfVectorizer from version pre-0.18 when using version 0.18. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/nihar/anaconda3/envs/Info256/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version pre-0.18 when using version 0.18. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/nihar/anaconda3/envs/Info256/lib/python3.5/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator FeatureUnion from version pre-0.18 wh

[[ 0.00297305  0.99702695]]


array([[ 0.00297305,  0.99702695]])